In [92]:
import pandas as pd
import numpy as np
import pulp
import matplotlib.pyplot as plt
import seaborn as sns

Sets and Parameters

SS
S: Set of students

CC
C: Set of courses

PsP_s
Ps​: Set of program-specific courses for student ss
s

Decision Variables

Xsc∈{0,1}X_{sc} \in \{0, 1\}
Xsc​∈{0,1}:


Xsc=1X_{sc} = 1
Xsc​=1 if student ss
s is assigned to course cc
c
Xsc=0X_{sc} = 0
Xsc​=0 otherwise




Utility Function
Usc=max⁡(10−ranksc,1)U_{sc} = \max(10 - \text{rank}_{sc}, 1) 
Usc​=max(10−ranksc​,1) 

Objective Function 

Maximize∑s∈S∑c∈CUsc⋅Xsc\text{Maximize} \sum_{s \in S} \sum_{c \in C} U_{sc} \cdot X_{sc}
Maximize∑s∈S​∑c∈C​Usc​⋅Xsc​ 


Constraints
1. Mandatory Course Constraint
∀s∈S,∀c∈Ps:c is mandatory\forall s \in S, \forall c \in P_s : c \text{ is mandatory}
∀s∈S,∀c∈Ps​:c is mandatoryXsc=1X_{sc} = 1
Xsc​=1
2. Elective Course Limit Constraint
∀s∈S\forall s \in S
∀s∈S\sum_{c \in P_s : c \text{ is elective}} X_{sc} = \text{required_electives}_s

3. Course Capacity Constraint
∀c∈C\forall c \in C
∀c∈C∑s∈SXsc≤capacityc\sum_{s \in S} X_{sc} \leq \text{capacity}_c
∑s∈S​Xsc​≤capacityc​
4. Binary Constraint
∀s∈S,∀c∈C\forall s \in S, \forall c \in C
∀s∈S,∀c∈CXsc∈{0,1}X_{sc} \in \{0, 1\}
Xsc​∈{0,1}
Interpretation

Maximizes total utility of course assignments
Ensures mandatory courses are assigned
Limits elective courses per student
Respects course capacity constraints

In [141]:
def load_data():
    """Load necessary CSV files for course matching."""
    # Load CSV files
    course_data = pd.read_csv('course.csv')
    student_data = pd.read_csv('student.csv')
    elective_capacity_data = pd.read_csv('elective_capacity.csv')
    elective_preference_data = pd.read_csv('elective_preference.csv')
    
    # Strip whitespace from column names
    course_data.columns = course_data.columns.str.strip()
    student_data.columns = student_data.columns.str.strip()
    elective_capacity_data.columns = elective_capacity_data.columns.str.strip()
    elective_preference_data.columns = elective_preference_data.columns.str.strip()
    
    # Strip whitespace from string columns
    for df in [course_data, student_data, elective_capacity_data, elective_preference_data]:
        for col in df.select_dtypes(include=['object']).columns:
            df[col] = df[col].str.strip()
    
    return course_data, student_data, elective_capacity_data, elective_preference_data

def optimize_course_matching():
    """
    Optimize course matching for students
    """
    # Load data
    course_data, student_data, elective_capacity_data, elective_preference_data = load_data()
    
    # Create PuLP model
    model = pulp.LpProblem("Course_Matching", pulp.LpMaximize)
    
    # Prepare data
    students = student_data['student_id'].tolist()
    courses = course_data['course_id'].tolist()
    
    # Decision variables
    # X[s,c] = 1 if student s is assigned to course c, 0 otherwise
    X = pulp.LpVariable.dicts("X", 
                             [(s, c) for s in students for c in courses], 
                             cat=pulp.LpBinary)
    
    # Objective function: utility based on preference ranking
    def calculate_utility(rank):
        return max(10 - rank, 1)
    
    # Preference utility
    preference_utility = []
    for _, pref in elective_preference_data.iterrows():
        student_id = pref['student_id']
        course_id = pref['course_id']
        utility = calculate_utility(pref['preference_rank'])
        preference_utility.append(utility * X[(student_id, course_id)])
    
    # Set objective: maximize preference utility
    model += pulp.lpSum(preference_utility), "Preference Utility"
    
    # Constraints
    # 1. Mandatory course constraints
    for _, student in student_data.iterrows():
        student_id = student['student_id']
        program_id = student['program_id']
        
        # Identify mandatory courses for this student's program
        mandatory_courses = course_data[
            (course_data['program_id'] == program_id) & 
            (course_data['mandatory'] == 1)
        ]['course_id'].tolist()
        
        # Ensure all mandatory courses are assigned
        for course_id in mandatory_courses:
            model += X[(student_id, course_id)] == 1, f"Mandatory_{student_id}_{course_id}"
    
    # 2. Elective course constraints
    for _, student in student_data.iterrows():
        student_id = student['student_id']
        program_id = student['program_id']
        required_electives = student['required_electives']
        
        # Identify elective courses for this student's program
        elective_courses = course_data[
            (course_data['program_id'] == program_id) & 
            (course_data['mandatory'] == 0)
        ]['course_id'].tolist()
        
        # Ensure exact number of electives are assigned
        if elective_courses:
            model += pulp.lpSum(X[(student_id, c)] for c in elective_courses) == required_electives, f"ElectiveLimit_{student_id}"
    
    # 3. Elective course capacity constraints
    for _, capacity in elective_capacity_data.iterrows():
        course_id = capacity['course_id']
        max_capacity = capacity['capacity']
        
        model += pulp.lpSum(X[(s, course_id)] for s in students) <= max_capacity, f"ElectiveCapacity_{course_id}"
    
    # Solve the model
    model.solve()
    
    # Check solution status
    if pulp.LpStatus[model.status] != 'Optimal':
        print("Could not find an optimal solution.")
        return None
    
    # Extract results
    results = []
    for _, student in student_data.iterrows():
        student_id = student['student_id']
        program_id = student['program_id']
        
        # Find mandatory courses
        mandatory_courses = course_data[
            (course_data['program_id'] == program_id) & 
            (course_data['mandatory'] == 1)
        ]
        
        # Find elective courses
        elective_courses = course_data[
            (course_data['program_id'] == program_id) & 
            (course_data['mandatory'] == 0)
        ]
        
        # Track assigned courses
        for _, course in mandatory_courses.iterrows():
            if X[(student_id, course['course_id'])].value() > 0.5:
                results.append({
                    'student_id': student_id,
                    'student_name': student_data[student_data['student_id'] == student_id]['name'].iloc[0],
                    'course_type': 'Mandatory',
                    'course_id': course['course_id'],
                    'course_name': course['course_name']
                })
        
        for _, course in elective_courses.iterrows():
            if X[(student_id, course['course_id'])].value() > 0.5:
                results.append({
                    'student_id': student_id,
                    'student_name': student_data[student_data['student_id'] == student_id]['name'].iloc[0],
                    'course_type': 'Elective',
                    'course_id': course['course_id'],
                    'course_name': course['course_name']
                })
    
    # Convert to DataFrame and export
    results_df = pd.DataFrame(results)
    results_df.to_csv('student_course_matching.csv', index=False)
    
    print("Course matching completed. Results saved to student_course_matching.csv")
    return results_df

def main():
    # Run optimization
    optimize_course_matching()

if __name__ == "__main__":
    main()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Applications/anaconda3/envs/python313/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/00/bhmglhlx427341n1cdxtx73w0000gn/T/8839ca0637c142c09070f15138124cb8-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/00/bhmglhlx427341n1cdxtx73w0000gn/T/8839ca0637c142c09070f15138124cb8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 35 COLUMNS
At line 308 RHS
At line 339 BOUNDS
At line 417 ENDATA
Problem MODEL has 30 rows, 77 columns and 98 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 133 - 0.00 seconds
Cgl0004I processed model has 5 rows, 15 columns (15 integer (15 of which binary)) and 15 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -133



Let S = {s₁, s₂, ..., sₙ} be the set of students
Let C = {c₁, c₂, ..., cₘ} be the set of courses
Let L = {l₁, l₂, ..., lₖ} be the set of lab sections
Let P = {(s, c, l, r) | s ∈ S, c ∈ C, l ∈ L, r ∈ ℕ} be the set of lab preferences

Decision Variables:

Y[s,l] ∈ {0, 1}

Y[s,l] = 1 if student s is assigned to lab section l
Y[s,l] = 0 otherwise

Preference Utility Function:
u(r) : ℕ → ℝ⁺
u(r) = max(10 - r, 1)
Where:

r is the preference ranking
u(r) transforms the ranking into a utility score

Objective Function:
max Z = ∑[s∈S, l∈L, c∈C] u(r[s,c,l]) * Y[s,l]
Constraints:

Lab Assignment Constraint:
∀s ∈ S, ∀c ∈ C with lab:
∑[l∈L(c)] Y[s,l] = 1
Lab Time Conflict Constraint:
∀s ∈ S, ∀(l₁, l₂) ∈ L with conflicting times:
Y[s,l₁] + Y[s,l₂] ≤ 1
Lab Capacity Constraint:
∀l ∈ L:
∑[s∈S] Y[s,l] ≤ capacity[l]

Detailed Description:

Objective Function Breakdown:

Maximizes the total utility of lab assignments
Calculates utility for each student-course-lab combination
Utility depends on the student's preference ranking
Higher preference (lower rank) gives higher utility
Minimum utility is 1, maximum is 9


Preference Utility Function [u(r)]: 
Transforms preference ranking into a utility score
For rank 1 (most preferred): u(1) = 9
For rank 10 or higher: u(r) = 1
Creates a non-linear utility scale that heavily rewards top preferences


Constraints Explanation: 
Lab Assignment: Ensures each course with a lab gets exactly one lab section per student
Time Conflict: Prevents assigning conflicting lab times to the same student
Capacity: Ensures no lab section exceeds its maximum capacity

In [142]:
def load_data():
    """
    Load necessary data for lab matching optimization
    """
    # Load CSV files
    student_course_matching = pd.read_csv('student_course_matching.csv')
    lab_time_data = pd.read_csv('lab_time.csv')
    day_data = pd.read_csv('day.csv')
    pre_lab_ele_man_data = pd.read_csv('pre_lab_ele_man.csv')
    theory_time_data = pd.read_csv('theory_time.csv')
    course_data = pd.read_csv('course.csv')
    
    # Strip whitespace from column names and data
    for df in [student_course_matching, lab_time_data, day_data, 
               pre_lab_ele_man_data, theory_time_data, course_data]:
        df.columns = df.columns.str.strip()
        for col in df.select_dtypes(include=['object']).columns:
            df[col] = df[col].str.strip()
    
    # Create day mapping
    day_mapping = dict(zip(day_data['id_day'], day_data['day']))
    
    return (student_course_matching, lab_time_data, day_mapping, 
            pre_lab_ele_man_data, theory_time_data, course_data)

def check_time_conflict(day1, start1, end1, day2, start2, end2):
    """
    Check if two time slots conflict with each other.
    
    Args:
        day1, day2: Days to compare
        start1, end1: Start and end times for first slot
        start2, end2: Start and end times for second slot
        
    Returns:
        True if there's a conflict, False otherwise
    """
    # If different days, no conflict
    if day1 != day2:
        return False
    
    # Convert times to integers for comparison
    try:
        start1 = int(start1)
        end1 = int(end1)
        start2 = int(start2)
        end2 = int(end2)
        
        # Check for overlap
        # No conflict if one ends before or at the same time the other starts 
        if end1 <= start2 or end2 <= start1:
            return False
            
        # Otherwise, there's a conflict
        return True
    except ValueError:
        # If conversion fails, assume no conflict
        return False

def optimize_lab_matching():
    """
    Optimize lab matching for students based on course matching and preferences
    """
    # Load data
    (student_course_matching, lab_time_data, day_mapping, 
     pre_lab_ele_man_data, theory_time_data, course_data) = load_data()
    
    # Print initial data for debugging
    print("Initial Data Analysis:")
    print("Total students in course matching:", len(student_course_matching['student_id'].unique()))
    print("Total lab time entries:", len(lab_time_data))
    
    # Prepare the optimization model with more relaxed constraints
    model = pulp.LpProblem("Lab_Matching", pulp.LpMaximize)
    
    # Create utility calculation function
    def calculate_utility(rank):
        return max(10 - rank, 1)
    
    # Prepare necessary data structures
    students = student_course_matching['student_id'].unique()
    
    # Prepare lab sections for courses with labs
    lab_time_data['lab_id'] = lab_time_data.apply(
        lambda x: f"{x['course_id']}-{x['lab']}", axis=1
    )
    
    # Decision variables
    # Y[s,l] = 1 if student s is assigned to lab section l, 0 otherwise
    Y = pulp.LpVariable.dicts("Y", 
        [(s, l) for s in students for l in lab_time_data['lab_id']], 
        cat=pulp.LpBinary
    )
    
    # Calculate lab preference utility
    lab_utility = []
    for _, pref in pre_lab_ele_man_data.iterrows():
        student_id = pref['student_id']
        course_id = pref['course_id']
        lab_num = pref['lab']
        lab_id = f"{course_id}-{lab_num}"
        
        # Check if the student is taking this course and it has a lab
        course_match = student_course_matching[
            (student_course_matching['student_id'] == student_id) & 
            (student_course_matching['course_id'] == course_id)
        ]
        
        # Check if the course has a lab
        course_has_lab = course_data[
            (course_data['course_id'] == course_id) & 
            (course_data['has_lab'] == 1)
        ]
        
        if not course_match.empty and not course_has_lab.empty and lab_id in lab_time_data['lab_id'].values:
            utility = calculate_utility(pref['preference_rank'])
            lab_utility.append(utility * Y[(student_id, lab_id)])
    
    # Set objective: maximize lab preference utility
    model += pulp.lpSum(lab_utility), "Lab Preference Utility"
    
    # Constraints
    # 1. Lab assignment constraints for courses with labs
    for student_id in students:
        # Get courses for this student
        student_courses = student_course_matching[
            student_course_matching['student_id'] == student_id
        ]
        
        for _, course in student_courses.iterrows():
            # Check if course has a lab
            course_has_lab = course_data[
                (course_data['course_id'] == course['course_id']) & 
                (course_data['has_lab'] == 1)
            ]
            
            if not course_has_lab.empty:
                # Find possible lab sections for this course
                possible_labs = lab_time_data[
                    lab_time_data['course_id'] == course['course_id']
                ]
                
                # Create lab IDs
                course_lab_ids = [
                    f"{course['course_id']}-{lab['lab']}" 
                    for _, lab in possible_labs.iterrows()
                ]
                
                # If the course has a lab, student must be assigned exactly one lab
                model += pulp.lpSum(Y[(student_id, l)] for l in course_lab_ids) == 1, \
                    f"LabAssignment_{student_id}_{course['course_id']}"
    
    # 2. Lab time conflict constraints
    lab_time_conflicts = []
    
    # Check for conflicts between lab times
    for student_id in students:
        # Get courses for this student with labs
        student_courses_with_labs = student_course_matching[
            (student_course_matching['student_id'] == student_id) & 
            (student_course_matching['course_id'].isin(
                course_data[course_data['has_lab'] == 1]['course_id']
            ))
        ]
        
        # Get lab times for this student's courses
        lab_schedules = []
        for _, course in student_courses_with_labs.iterrows():
            # Find lab times for this course
            course_labs = lab_time_data[lab_time_data['course_id'] == course['course_id']]
            
            for _, lab in course_labs.iterrows():
                lab_schedules.append({
                    'course_id': lab['course_id'],
                    'lab': lab['lab'],
                    'day': day_mapping.get(lab['id_day'], 'Unknown'),
                    'start_time': lab['start_time'],
                    'end_time': lab['end_time']
                })
        
        # Check for conflicts between all lab pairs
        for i in range(len(lab_schedules)):
            for j in range(i+1, len(lab_schedules)):
                lab1 = lab_schedules[i]
                lab2 = lab_schedules[j]
                
                if check_time_conflict(
                    lab1['day'], lab1['start_time'], lab1['end_time'],
                    lab2['day'], lab2['start_time'], lab2['end_time']
                ):
                    lab_id1 = f"{lab1['course_id']}-{lab1['lab']}"
                    lab_id2 = f"{lab2['course_id']}-{lab2['lab']}"
                    lab_time_conflicts.append((student_id, lab_id1, lab_id2))
    
    # Add constraints to prevent lab time conflicts
    for student_id, lab_id1, lab_id2 in lab_time_conflicts:
        model += Y[(student_id, lab_id1)] + Y[(student_id, lab_id2)] <= 1, \
            f"LabTimeConflict_{student_id}_{lab_id1}_{lab_id2}"
    
    # Solve the model
    model.solve()
    
    # Print solver status and details
    print("\nSolver Status:", pulp.LpStatus[model.status])
    
    # Check solution status
    if pulp.LpStatus[model.status] not in ['Optimal', 'Feasible']:
        print("Could not find a solution. Analyzing potential issues...")
        return None
    
    # Prepare results
    results = []
    for student_id in students:
        # Get student's courses
        student_courses = student_course_matching[
            student_course_matching['student_id'] == student_id
        ]
        
        # Find student's name
        student_name = student_courses['student_name'].iloc[0]
        
        for _, course in student_courses.iterrows():
            # Find corresponding theory time
            theory_time = theory_time_data[theory_time_data['course_id'] == course['course_id']]
            
            # Prepare lab information
            lab_day = 'N/A'
            lab_start_time = 'N/A'
            lab_end_time = 'N/A'
            
            # Check if course has a lab
            course_info = course_data[course_data['course_id'] == course['course_id']]
            has_lab = course_info['has_lab'].iloc[0] if not course_info.empty else 0
            
            # If course has a lab, find lab information
            if has_lab == 1:
                # Find lab times for this course
                course_labs = lab_time_data[lab_time_data['course_id'] == course['course_id']]
                
                for _, lab in course_labs.iterrows():
                    lab_id = f"{lab['course_id']}-{lab['lab']}"
                    
                    # Check if this lab is assigned to the student
                    if Y[(student_id, lab_id)].value() > 0.5:
                        lab_day = day_mapping.get(lab['id_day'], 'Unknown')
                        lab_start_time = lab['start_time']
                        lab_end_time = lab['end_time']
                        break
            
            # Append result for this course
            results.append({
                'student_id': student_id,
                'student_name': student_name,
                'course_id': course['course_id'],
                'course_name': course['course_name'],
                'course_type': course['course_type'],
                'theory_day': day_mapping.get(theory_time['id_day'].iloc[0], 'Unknown') if not theory_time.empty else 'N/A',
                'theory_start_time': theory_time['start_time'].iloc[0] if not theory_time.empty else 'N/A',
                'theory_end_time': theory_time['end_time'].iloc[0] if not theory_time.empty else 'N/A',
                'lab_day': lab_day,
                'lab_start_time': lab_start_time,
                'lab_end_time': lab_end_time
            })
    
    # Convert to DataFrame and export
    results_df = pd.DataFrame(results)
    results_df.to_csv('student_lab_matching.csv', index=False)
    
    print("Course matching completed. Results saved to student_lab_matching.csv")
    return results_df

def main():
    # Run lab optimization
    optimize_lab_matching()

if __name__ == "__main__":
    main()

Initial Data Analysis:
Total students in course matching: 7
Total lab time entries: 18
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Applications/anaconda3/envs/python313/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/00/bhmglhlx427341n1cdxtx73w0000gn/T/66259f15bc314921b5433f435e6f94ca-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/00/bhmglhlx427341n1cdxtx73w0000gn/T/66259f15bc314921b5433f435e6f94ca-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 24 COLUMNS
At line 172 RHS
At line 192 BOUNDS
At line 231 ENDATA
Problem MODEL has 19 rows, 38 columns and 38 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 153 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at